Importing the required packages

In [1]:
import spacy
import pdfminer
import re
import os
import pandas as pd

Imported the code inside the pdf2text file

In [2]:
import pdf2txt

Converting PDFs to text

In [3]:
def convert_pdf(f):
    output_filename = os.path.basename(os.path.splitext(f)[0]) + '.txt'
    output_filepath = os.path.join('output/txt/', output_filename)
    pdf2txt.main(args=[f, "--outfile", output_filepath])
    print(output_filepath + " saved successfully!!!")
    return open(output_filepath).read()

Loading the NLP language model

In [4]:
nlp = spacy.load('en_core_web_sm')

Defined a data structure which has the data fields needed

In [5]:
result_dict = {
    'name': [],
    'phone': [],
    'email': [],
    'skills': []
}

names = []
phones = []
emails = []
skills = []

Wrote a function that parses the content of a text file

In [ ]:
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})")
    doc = nlp(text)
    name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0]
    print(name)
    email = [word for word in doc if word.like_email == True][0]
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset, text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully")